크롤링을 이용한 서울시 스타벅스 매장 목록 데이터 생성

1. 크롤링 실행하기 위해 selenium 라이브러리의 webdriver 불러옴
2. 크롤링으로 가져온 HTML에서 정보를 추출하기 위해 BeautifulSoup 라이브러리 추가
3. 추출된 데이터를 엑셀로 저장하기 위해 pandas라이브러리 추가

In [1]:
# 라이브러리 임포트
!pip install selenium==3.141

from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

webdriver.Chrome함수를 이용해 크롬 브라우저 실행 후 driver 변수에 할당

driver.get(url) 명령어를 통해 해당 url로 이동

크롬 브라우저가 자동으로 실행됨

In [2]:
#webdriver 실행 후 스타벅스의 지역별 매장 검색 화면에 접속
driver = webdriver.Chrome('c:/jin/chromedriver.exe')
# driver = webdriver.Chrome('../chromedriver')
url = 'https://www.istarbucks.co.kr/store/store_map.do?disp=locale'
driver.get(url)

In [3]:
#webdriver로 ‘서울’ 버튼 요소를 찾아 클릭
seoul_btn = '#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a'
driver.find_element_by_css_selector(seoul_btn).click()

In [4]:
# webdriver로 ‘전체’ 버튼 요소를 찾아 클릭
all_btn = '#mCSB_2_container > ul > li:nth-child(1) > a'
driver.find_element_by_css_selector(all_btn).click()

In [5]:
# BeautifulSoup으로 HTML 파서 만들기
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [6]:
#  select()를 이용해 원하는 HTML 태그를 모두 찾아오기
starbucks_soup_list = soup.select('li.quickResultLstCon')
print(len(starbucks_soup_list))

578


In [7]:
#  태그 문자열 살펴보기
starbucks_soup_list[0]


<li class="quickResultLstCon" data-code="3762" data-hlytag="null" data-index="0" data-lat="37.501087" data-long="127.043069" data-name="역삼아레나빌딩" data-storecd="1509" style="background:#fff"> <strong data-my_siren_order_store_yn="N" data-name="역삼아레나빌딩" data-store="1509" data-yn="N">역삼아레나빌딩  </strong> <p class="result_details">서울특별시 강남구 언주로 425 (역삼동)<br/>1522-3232</p> <i class="pin_general">리저브 매장 2번</i></li>

In [8]:
# 스타벅스 매장 정보 샘플 확인
startbucks_store = starbucks_soup_list[0]
name = startbucks_store.select('strong')[0].text.strip()
lat = startbucks_store['data-lat'].strip()
lng = startbucks_store['data-long'].strip()
store_type = startbucks_store.select('i')[0]['class'][0][4:]
address = str(startbucks_store.select('p.result_details')[0]).split('<br/>')[0].split('>')[1]
tel = str(startbucks_store.select('p.result_details')[0]).split('<br/>')[1].split('<')[0]

print(name)         # 매장명
print(lat)          # 위도
print(lng)          # 경도
print(store_type)   # 매장 타입
print(address)      # 주소
print(tel)          # 전화번호

역삼아레나빌딩
37.501087
127.043069
general
서울특별시 강남구 언주로 425 (역삼동)
1522-3232


In [9]:
#  서울시 스타벅스 매장 목록 데이터 만들기
starbucks_list = []
for item in starbucks_soup_list:
    name = item.select('strong')[0].text.strip();
    lat = item['data-lat'].strip()
    lng = item['data-long'].strip()
    store_type = item.select('i')[0]['class'][0][4:]
    address = str(item.select('p.result_details')[0]).split('<br/>')[0].split('>')[1]
    tel = str(item.select('p.result_details')[0]).split('<br/>')[1].split('<')[0]
    
    starbucks_list.append( [ name, lat, lng, store_type, address, tel])

In [10]:
#  pandas의 데이터프레임 생성
columns = ['매장명','위도','경도','매장타입', '주소','전화번호']
seoul_starbucks_df = pd.DataFrame(starbucks_list, columns = columns)
seoul_starbucks_df.head()

,매장명,위도,경도,매장타입,주소,전화번호
0,역삼아레나빌딩,37.501087,127.043069,general,서울특별시 강남구 언주로 425 (역삼동),1522-3232
1,논현역사거리,37.510178,127.022223,general,서울특별시 강남구 강남대로 538 (논현동),1522-3232
2,신사역성일빌딩,37.514132,127.020563,general,서울특별시 강남구 강남대로 584 (논현동),1522-3232
3,국기원사거리,37.499517,127.031495,general,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232
4,대치재경빌딩R,37.494668,127.062583,reserve,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232


In [11]:
# 데이터프레임의 요약 정보 확인
seoul_starbucks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 578 entries, 0 to 577
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   매장명     578 non-null    object
 1   위도      578 non-null    object
 2   경도      578 non-null    object
 3   매장타입    578 non-null    object
 4   주소      578 non-null    object
 5   전화번호    578 non-null    object
dtypes: object(6)
memory usage: 27.2+ KB


In [12]:
# 엑셀로 저장
seoul_starbucks_df.to_excel('seoul_starbucks_list.xlsx', index=False)